# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification

## Step2 加载数据

In [2]:
import pandas as pd

data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


## Step3 创建Dataset

In [4]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [5]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step4 划分数据集

In [6]:
from torch.utils.data import random_split


trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [7]:
for i in range(10):
    print(trainset[i])

('离外滩五分钟步行即到，而且正对着外滩观光隧道，想到东方明珠和金茂游玩比较方便，去南京路也相对近。这次订的是没有窗户的大床房。房间空间较小，但尚算可以接受，因为只住一晚的缘故。但酒店的空调和热水壶都出现故障，上海的节能工作做得真是好，连基本的空调环境都达不到。住在房间里不知天黑天白，因为没有窗户，所以第二天几乎睡到十点，要早起的朋友一定要记得闹铃。洗手间是整体浴室，空间刚刚好，洗手台面比较小，其他尚算可以接受。总的来说，中规中矩。如果喜欢住在外滩附近的朋友可以考虑选择。还有一点值得表扬的，楼层设置了公用电脑可以上网的。提供了方便。', 1)
('酒店人员的服务相当好，几乎是有求必应，在得知我们飞机晚点后，一直派专人守在门口。由于给到我们的房间与我们的要求有出入，酒店人员立即想办法帮我们解决，听说是他们的财务经理把房间让给了我们，让我们很感动，房间的价钱贵了点，但物有所值，下次去九寨沟还住喜来登。这次汶川地震应该没有影响到他们吧，祝一切平安！', 1)
('就在长途汽车站附近，步行10分钟吧。就是马路路口斜对过。在闹市区，路口有欧什么的西餐便餐店，环境不错。宾馆就是标准的4星，挺好的。附近还有kfc和农工商超市。', 1)
('外出时一般都会先看看当地的如家。这家店挺不错的。性价比高，服务也足够周到仔细，反应也很快。不过不怎么喜欢他们的制服啦，花花的，很难看。对了，只是离车站不是很近，边上那家超市，食品怀疑过期(面包)，离恐龙园也远，但是门口就有公交站台，还算交通方便。补充点评2008年1月28日：说错了，是虎丘，哈哈~~', 1)
('房间在当地还不错。早餐一般般。周边环境不错，有鸡骚扰。', 1)
('海底世界酒店整体环境不错，一个很雅致的庭院，比较安静，且在亚龙湾算是个相对便宜的酒店，我住的是豪华单人间，2米的大床，房间也很宽敞，还有个阳台，与环球城酒店紧挨着，服务也挺好，由于没有电梯，入住及离开时都是服务人员主动帮忙搬运行李，态度很好，只有一点不太满意，就是酒店提供的海底世界半潜观光船游览买一送一服务，我是想和老人及一个不满三岁的孩子去看，前台告诉我孩子可以免费，但去买票时小孩还需半价100元，感觉不值就放弃了。', 1)
('也许真是从三亚回来，觉得4星的标准真的差距很明显！酒店的房间很干净，就是设施有点陈旧了，很多地方看上去比较“烂”。提供的设施也很一般

## Step5 创建Dataloader

In [8]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./model")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './model'.

In [ ]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [ ]:
next(enumerate(validloader))[1]

## Step6 创建模型及优化器

In [ ]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("./model")

if torch.cuda.is_available():
    model = model.cuda()

In [ ]:
model = torch.nn.DataParallel(model)

In [ ]:
# 用于测试模型同步所消耗时间

In [ ]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [ ]:
import time

def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        start = time.time()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            loss = output.loss.mean()
            loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {loss.mean().item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}, time: {time.time() - start}")

## Step8 模型训练

In [ ]:
train()

## Step9 模型预测

In [ ]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

In [ ]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [ ]:
pipe(sen)